In [3]:
import os
import copy
import csv
import glob
import pickle
import matplotlib.pyplot as plt
from collections import Counter
import open3d as o3d
import numpy as np
import nibabel as nib
import numpy as np
from scipy import ndimage
from skimage import measure
import ants
import k3d

%matplotlib inline

def marching_cubes_3d(data):
  vertices, triangles, _, _ = measure.marching_cubes(data, level=0)
  return vertices, triangles

def voxel_to_mesh(data, flip=True):
  vertices, triangles, _, _ = measure.marching_cubes(data, level=0)
  mesh = o3d.geometry.TriangleMesh()
  mesh.vertices = o3d.utility.Vector3dVector(vertices)
  if flip:
    tri0 = triangles.copy()
    triangles[:, 0] = tri0[:, 2]
    triangles[:, 2] = tri0[:, 0]
  mesh.triangles = o3d.utility.Vector3iVector(triangles)
  mesh.compute_vertex_normals()
  
  return mesh

def voxel_to_bd_pcd(voxel, threshold = 8):
  kernel = np.ones((2,2,2))
  voxel_bd = ndimage.convolve((voxel).astype(int), kernel, mode='constant', cval=0.0)
  voxel_v = np.argwhere((voxel_bd<threshold)&(voxel_bd>0)).astype(float) + 0.5
  voxel_pcd = o3d.geometry.PointCloud()
  voxel_pcd.points = o3d.utility.Vector3dVector(voxel_v)
  return voxel_pcd

def pt_to_tex(boundary_pt, aseg, peri_list = [1.5, 0.5, -0.5, -1.5]):
  
	texture = np.zeros(boundary_pt.shape)
	# print(texture.shape)
  
	for n in range(boundary_pt.shape[0]):
		a= []
		x, y, z = (boundary_pt[n][0]), (boundary_pt[n][1]), (boundary_pt[n][2])
		
		for i in peri_list:
			for j in peri_list:
				for k in peri_list:
					tex = aseg[int(x+i),int(y+j),int(z+k)]
					if tex!=0:
						a.append(tex)
		if a:
			counts = Counter(a)
			most_common_value = max(counts, key=lambda x: counts[x] if (x!=0) else -1)
			texture[n] = int(most_common_value)

	return  texture

# Generate dataset

In [4]:
# Template voxel data
draw = True
seg_list = glob.glob(rf"../test_data/*/synthseg.nii.gz")
cuttail_cuttail = np.load(rf'../temp_meshes/lv_hippo_cuttail_cuttail.npy')

fixed = ants.from_numpy(cuttail_cuttail)
fixed_mesh = voxel_to_mesh(cuttail_cuttail)

fixed_mesh_tri = np.asarray(fixed_mesh.triangles)
tri0=fixed_mesh_tri.copy()
fixed_mesh_tri[:,[0,1]] = tri0[:,[1,0]]
fixed_mesh.paint_uniform_color([1.0,0.5,0.5])

for filename in seg_list:
    print(filename)
    id = filename.split('/')[-2]
    seg = np.asarray(nib.load(filename).get_fdata())
    aseg_peri = seg.copy()
    seg[seg==5]=4 # left LV 
    seg[seg==44]=43 # right LV
    seg [(seg!=4)&(seg!=43)&(seg!=17)&(seg!=53)]=0 

    aseg_peri[(aseg_peri!=43)&(aseg_peri!=11)&(aseg_peri!=10)]=0

    aseg_peri[aseg_peri==10]=1 # Thalamus
    aseg_peri[aseg_peri==11]=2 # Caudate
    aseg_peri[aseg_peri==43]=3 # Opposite LV

    moving = ants.from_numpy(seg)
    moving_aseg = ants.from_numpy(aseg_peri)

    transformR = ants.registration(fixed=fixed, moving=moving, type_of_transform='Rigid')
    warpedR = ants.apply_transforms(fixed=fixed, moving=moving, transformlist=transformR['fwdtransforms'], interpolator= "nearestNeighbor").numpy()
    warpedaseg = ants.apply_transforms(fixed=fixed, moving=moving_aseg, transformlist=transformR['fwdtransforms'], interpolator= "nearestNeighbor").numpy()

    # save ants image
    np.save(filename.replace("synthseg.nii.gz","warpedaseg.npy"), warpedaseg)
    np.save(filename.replace("synthseg.nii.gz","warpedR.npy"), warpedR)

    pcd_lv = voxel_to_bd_pcd((warpedR==4).astype(int))
    pcd_hippo = voxel_to_bd_pcd((warpedR==17).astype(int))

    texture = pt_to_tex(np.asarray(pcd_lv.points), warpedaseg)
    pcd_lv.paint_uniform_color([1.0, 0.5, 0.5])
    pcd_hippo.paint_uniform_color([0.5, 0.5, 1.0])

    pcd_lv.colors= pcd_lv.colors = o3d.utility.Vector3dVector(texture/3-0.1)
    o3d.visualization.draw_geometries([pcd_lv, pcd_hippo, fixed_mesh])

    pts= np.asarray(pcd_lv.points)
    tex1_index = np.unique(np.argwhere(texture==1)[:,0])
    tex2_index = np.unique(np.argwhere(texture==2)[:,0])
    tex3_index = np.unique(np.argwhere(texture==3)[:,0])
    pt1 = pts[tex1_index]
    pt2 = pts[tex2_index]
    pt3 = pts[tex3_index]
    if draw:
        plot = k3d.plot()
        pt1_obj = k3d.points(pt1, point_size=1, color=0xff8080) 
        pt2_obj = k3d.points(pt2, point_size=1, color=0x8080ff)
        pt3_obj = k3d.points(pt3, point_size=1, color=0x80ff80)
        hippo_obj = k3d.points(np.asarray(pcd_hippo.points), point_size = 0.5, color=0x80ffff) 
        lv_obj = k3d.points(np.asarray(pcd_lv.points), point_size = 0.5, color=0xff80ff) 

        plot += pt1_obj
        plot += pt2_obj
        plot += pt3_obj
        plot += hippo_obj
        plot += lv_obj

        plot.display()                                                                                                              
    # Save Hippo_L optimization data
    data ={}
    data['lv'] = np.asarray(pcd_lv.points)
    data['hippo']=np.asarray(pcd_hippo.points)
    data['tex1'] = pt1
    data['tex2'] = pt2
    data['tex3'] = pt3
    with open (filename.replace("synthseg.nii.gz","optim_data.pkl"), 'wb') as f:
        pickle.dump(data, f)



../test_data/test2/synthseg.nii.gz
[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


/opt/conda/envs/pytorch3d/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

../test_data/test1/synthseg.nii.gz
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


Output()